In [1]:
from collections import deque
from enum import Enum
from itertools import combinations

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from collections import Counter, defaultdict

from mesa import Model, Agent


class Move(Enum):
    COOPERATE = 1
    DEFECT = 2


class AxelrodAgent(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.points = 0

    def step(self):
        """
        the move and any logic for deciding
        on the move goes here

        Returns
        -------
        Move.COOPERATE or Move.DEFECT

        """
        raise NotImplemetedError

    def receive_payoff(self, payoff, my_move, opponent_move):
        """

        Parameters
        ----------
        payoff : int
        my_move : {Move.COOPERATE, Move.DEFECT}
        opponents_move : {Move.COOPERATE, Move.DEFECT}

        """
        self.points += payoff
        
    def reset(self):
        """
        called after playing N iterations against
        another player
        """
        raise NotImplementedError
        

class TitForTat(AxelrodAgent):
    
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.opponent_last_move = Move.COOPERATE
    
    def step(self):
        return self.opponent_last_move
    
    def receive_payoff(self, payoff, my_move, opponent_move):
        super().receive_payoff(payoff, my_move, opponent_move)
        self.opponent_last_move = opponent_move
        
    def reset(self):
        self.opponent_last_move = Move.COOPERATE


class ContriteTitForTat(AxelrodAgent):
    
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.opponent_last_two_moves = deque([Move.COOPERATE, Move.COOPERATE], maxlen=2)
    
    def step(self):
        if (self.opponent_last_two_moves[0] == Move.DEFECT) and\
           (self.opponent_last_two_moves[1] == Move.DEFECT):
            return Move.DEFECT
        else:
            return Move.COOPERATE
    
    def receive_payoff(self, payoff, my_move, opponent_move):
        super().receive_payoff(payoff, my_move, opponent_move)
        self.opponent_last_two_moves.append(opponent_move)
        
    def reset(self):
        self.opponent_last_two_moves = deque([Move.COOPERATE, Move.COOPERATE], maxlen=2)        

        
class Defector(AxelrodAgent):
    def step(self):
        return Move.DEFECT
    
    def reset(self):
        pass
    
class Cooperator(AxelrodAgent):
    def step(self):
        return Move.COOPERATE
    
    def reset(self):
        pass
    
class GrimTrigger(AxelrodAgent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.betrayed = False
        
    def step(self):
        if self.betrayed:
            return Move.DEFECT
        else:
            return Move.COOPERATE
        
    def receive_payoff(self, payoff, my_move, opponent_move):
        super().receive_payoff(payoff, my_move, opponent_move)
        if opponent_move == Move.DEFECT:
            self.betrayed = True
                
    def reset(self):
        self.betrayed = False
        
class Pavlov(AxelrodAgent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.my_last_move = Move.COOPERATE
        self.opponent_last_move = Move.COOPERATE
            
    def step(self):
        if self.my_last_move == self.opponent_last_move:
            return self.my_last_move
        else:
            return self.opponent_last_move
    
    def receive_payoff(self, payoff, my_move, opponent_move):
        super().receive_payoff(payoff, my_move, opponent_move)
        self.my_last_move = my_move
        self.opponent_last_move = opponent_move
            
        
    def reset(self):
        self.my_last_move = Move.COOPERATE
        self.opponent_last_move = Move.COOPERATE
        
class Random(AxelrodAgent):
    
    def step(self):
        if self.random.random() < 0.5:
            return Move.COOPERATE
        else:
            return Move.DEFECT
        
    def reset(self):
        pass
        
class AxelrodModel(Model):
    """A model with some number of agents."""
    def __init__(self, N, seed, noise_level):
        super().__init__(seed=seed)
        self.noise_level = noise_level
        self.num_iterations = N
        self.agents = []
        self.payoff_matrix = {}
        
        self.payoff_matrix[(Move.COOPERATE, Move.COOPERATE)] = (2, 2)
        self.payoff_matrix[(Move.COOPERATE, Move.DEFECT)] = (0, 3)
        self.payoff_matrix[(Move.DEFECT, Move.COOPERATE)] = (3, 0)
        self.payoff_matrix[(Move.DEFECT, Move.DEFECT)] = (1, 1)
        
        # Create agents
        for i, agent_class in enumerate(AxelrodAgent.__subclasses__()):
            a = agent_class(i, self)
            self.agents.append(a)

    def step(self):
        """Advance the model by one step."""
        for agent_a, agent_b in combinations(self.agents, 2):
            for _ in range(self.num_iterations):
                move_a = agent_a.step()
                move_b = agent_b.step()
                
                if self.random.random() > self.noise_level:
                    if move_a == Move.COOPERATE:
                        move_a = Move.DEFECT
                    else:
                        move_a = Move.COOPERATE
                        
                if self.random.random() > self.noise_level:
                    if move_b == Move.COOPERATE:
                        move_b = Move.DEFECT
                    else:
                        move_b = Move.COOPERATE    
              
                payoff_a, payoff_b = self.payoff_matrix[(move_a, move_b)]
                
                agent_a.receive_payoff(payoff_a, move_a, move_b)
                agent_b.receive_payoff(payoff_b, move_b, move_a)
            agent_a.reset()
            agent_b.reset()
            
class EvolutionaryAxelrodModel(Model):
    """A model with some number of agents."""
    def __init__(self, num_agents, N, seed=None):
        super().__init__(seed=seed)
        
        self.num_iterations = N
        self.agents = []
        self.payoff_matrix = {}
        self.population_size = len(AxelrodAgent.__subclasses__())*num_agents
        
        self.payoff_matrix[(Move.COOPERATE, Move.COOPERATE)] = (2, 2)
        self.payoff_matrix[(Move.COOPERATE, Move.DEFECT)] = (0, 3)
        self.payoff_matrix[(Move.DEFECT, Move.COOPERATE)] = (3, 0)
        self.payoff_matrix[(Move.DEFECT, Move.DEFECT)] = (1, 1)        
        
        strategies = AxelrodAgent.__subclasses__()
        num_strategies = len(strategies)
        proportions = {agent_class:1/num_strategies for agent_class in strategies}

        self.num_iterations = N
        self.scores = defaultdict(list)
        for agent_class in strategies:
            self.scores[agent_class].append(proportions[agent_class])        
        
        self.initial_population_size = num_agents * num_strategies
        self.agent_id = 0
        self.build_population(proportions)

    
    def step(self):
        """Advance the model by one step."""
        for agent_a, agent_b in combinations(self.agents, 2):
            for _ in range(self.num_iterations):
                move_a = agent_a.step()
                move_b = agent_b.step()
                
                payoff_a, payoff_b = self.payoff_matrix[(move_a, move_b)]
                
                agent_a.receive_payoff(payoff_a, move_a, move_b)
                agent_b.receive_payoff(payoff_b, move_b, move_a)
            agent_a.reset()
            agent_b.reset()
        
        # calculate scores per class of agents
        scores = Counter()
        for agent in self.agents:
            scores[agent.__class__] += agent.points
            
        # normalize scores on unit interval
        proportions = {}
        
        proportions = {k: v*(v/sum(scores.values())) for k, v in scores.items()}
        
        # keep track of proportions over the generations
        for agent_class in AxelrodAgent.__subclasses__():
            self.scores[agent_class].append(proportions[agent_class])
        
        self.build_population(proportions)
        
    def build_population(self, proportions):
        """build the new population

        Parameters
        ----------
        proportions : dict
                      key is agent class, value is float

        """
        
        # build new population
        population = []

        # create a number of agents proportional to the normalized scores
        # ensure that the total size of the population (num_agents * num_strategies) 
        # stays as close to the initial population size
        
        for k, v in proportions.items():
            total_agents = round(v*self.initial_population_size)
            agent_class = k
            for _ in range(total_agents):
                self.agent_id += 1
                a = agent_class(self.agent_id, self)
                population.append(a)
        
        self.agents = population

In [ ]:
model = EvolutionaryAxelrodModel(10, 200)

for _ in range(50):
    model.step()

# visualizing results using matplotlib
fig, ax = plt.subplots()
for k, v in model.scores.items():
    ax.plot(v, label=k.__name__)
ax.legend()
plt.show()